In [1]:
print('hello')

hello


In [2]:
%pip install selenium
%pip install webdriver-manager
%pip install undetected-chromedriver
%pip install beautifulsoup4
%pip install pandas
%pip install tqdm
%pip install requests
%pip install pillow
%pip install colorama
%pip install IPython
%pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

In [3]:
# standard python
import os
import json
import time
import random
import warnings
import csv
from datetime import datetime

# external libs
import pandas as pd
import requests
from tqdm.notebook import tqdm

# beautiful soup
from bs4 import BeautifulSoup

# selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
# selenium with undetected tracking
import undetected_chromedriver as uc

# better visualization in terminal
from IPython.display import display, Image, HTML, clear_output
from PIL import Image as PILImage
from colorama import Fore, Style, init

# no warnings
warnings.filterwarnings("ignore", message="Pyarrow will become a required dependency")


/Users/enzoberreur/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# 0 - Webscrapping demo

## A - Functions

In [4]:
# type with human effect function (just for style)
def type_with_human_effect(element, text, min_delay=0.05, max_delay=0.15):
    """texting with random time between keys so that it's almost like a human (only for display)"""
    for char in text:
        element.send_keys(char)
        time.sleep(random.uniform(min_delay, max_delay))

# scrolling function
def smooth_scroll(driver, scroll_to=None, duration=2, steps=25):
    """scrolling the page"""
    # continue until the end of the page
    if scroll_to is None:
        total_height = driver.execute_script("return document.body.scrollHeight")
    else:
        total_height = scroll_to
    
    current_position = driver.execute_script("return window.pageYOffset")
    step_size = (total_height - current_position) / steps
    
    for i in range(steps):
        new_position = current_position + step_size * (i + 1)
        driver.execute_script(f"window.scrollTo(0, {new_position})")
        time.sleep(duration / steps)

# screenshot function
def take_and_display_screenshot(driver, filename="screenshot.png", width=800):
    """take a screenshot and show it in the notebook"""
    driver.save_screenshot(filename)
    
    # resizing the image to display it
    img = PILImage.open(filename)
    wpercent = (width / float(img.size[0]))
    hsize = int((float(img.size[1]) * float(wpercent)))
    img = img.resize((width, hsize), PILImage.LANCZOS)
    img.save(filename)
    
    # display image
    display(Image(filename=filename, width=width))

# highlight elements
def highlight_element(driver, element, duration=2):
    """highlights an element in the page to better follow (only for display)"""
    original_style = element.get_attribute("style")
    driver.execute_script("""
    arguments[0].setAttribute('style', arguments[1] + 
        '; border: 2px solid red; background: yellow; color: black;');
    """, element, original_style)
    time.sleep(duration)
    driver.execute_script("""
    arguments[0].setAttribute('style', arguments[1]);
    """, element, original_style)


## B - Example of Browser - Filling out a form

In [ ]:
# colorama init for terminal
init()

# setting up the browser
print(f"{Fore.CYAN}browser configuration...{Style.RESET_ALL}")

# compatible options with undetected browser
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")  # full screen window
options.add_argument("--disable-notifications")  # no notifs

# progress bar
for i in tqdm(range(10), desc="Webdriver init"):
    time.sleep(0.2)

# webdriver init with undetected lib (to avoid robots warnings from google)
driver = webdriver.Chrome(options=options)

print(f"\n{Fore.CYAN}📝 Filling out a Form{Style.RESET_ALL}")
driver.get("https://httpbin.org/forms/post")
time.sleep(2)
take_and_display_screenshot(driver, "form_initial.png")

print("...")

# Remplir les champs du formulaire avec un effet visuel
try:
    # Nom du client
    customer_name = driver.find_element(By.NAME, "custname")
    highlight_element(driver, customer_name)
    type_with_human_effect(customer_name, "Amaury Gellé")
    
    # Téléphone
    phone = driver.find_element(By.NAME, "custtel")
    highlight_element(driver, phone)
    type_with_human_effect(phone, "0123456789")
    
    # Email
    email = driver.find_element(By.NAME, "custemail")
    highlight_element(driver, email)
    type_with_human_effect(email, "amaurygelle@gmail.com")
    
    # Choix de pizza
    pizza_choice = driver.find_element(By.CSS_SELECTOR, "input[value='medium']")
    highlight_element(driver, pizza_choice)
    pizza_choice.click()
    
    # Garniture
    toppings = driver.find_element(By.NAME, "topping")
    highlight_element(driver, toppings)
    driver.execute_script("arguments[0].value = 'mushroom';", toppings)
    
    # Heure de livraison
    time_elem = driver.find_element(By.NAME, "delivery")
    highlight_element(driver, time_elem)
    type_with_human_effect(time_elem, "18:30")
    
    # Instructions
    instructions = driver.find_element(By.NAME, "comments")
    highlight_element(driver, instructions)
    type_with_human_effect(instructions, "Please leave it in front of the doorway - thanks!")
    
    take_and_display_screenshot(driver, "form_filled.png")
    
    # Soumission du formulaire
    print("Soumission du formulaire...")
    submit_button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
    highlight_element(driver, submit_button)
    submit_button.click()
    
    time.sleep(2)
    take_and_display_screenshot(driver, "form_submitted.png")
    
except Exception as e:
    print(f"Erreur lors du remplissage du formulaire: {e}")

## C - Mimicking a Google search

In [ ]:
# colorama init for terminal
init()

# setting up the browser
print(f"{Fore.CYAN}browser configuration...{Style.RESET_ALL}")

# compatible options with undetected browser
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")  # full screen window
options.add_argument("--disable-notifications")  # no notifs

# progress bar
for i in tqdm(range(10), desc="Webdriver init"):
    time.sleep(0.2)

# webdriver init with undetected lib (to avoid robots warnings from google)
driver = webdriver.Chrome(options=options)

print(f"{Fore.GREEN}Interactive Google Search{Style.RESET_ALL}")
print("Opening Google and accepting cookies...")

# Ouverture de Google
driver.get("https://www.google.com")
time.sleep(2)

# Accept cookies if we have to (might have been done in the past)
try:
    cookie_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Accept all')]"))
    )
    cookie_button.click()
    print("Cookies accepted ✓")
    time.sleep(1)
except:
    print("No cookies found or cookies already accepted")

# Typing with human-like behavior
search_box = driver.find_element(By.NAME, "q")
highlight_element(driver, search_box)
search_term = "Albert School data courses"
print(f"Looking: '{search_term}'")

type_with_human_effect(search_box, search_term)
take_and_display_screenshot(driver, "google_search_typing.png")

# Submitting the request
search_box.send_keys(Keys.RETURN)
time.sleep(3)
take_and_display_screenshot(driver, "google_results.png")

# Scrolling the page and taking screenshots
print("Scrolling the page...")
smooth_scroll(driver, duration=4)
take_and_display_screenshot(driver, "google_results_scrolled.png")

# Extracting data
print(f"{Fore.BLUE} Extracting links and descriptions...{Style.RESET_ALL}")

# Waiting for the elements of the page to have loaded
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='https://']"))
    )
    print("Loaded ✓")
except:
    print("Timeout for research terms")

# collect search results
search_results = []
time.sleep(1)

# Look for all elements
links = driver.find_elements(By.CSS_SELECTOR, "a[href^='https://']:not([href*='google'])")

print(f"Links found: {len(links)}")

for link in links:
    try:
        # get the URL
        url = link.get_attribute("href")
        
        # Get the text of the link
        title = link.text
        
        # if there is no title, get its parent element 
        if not title.strip():
            try:
                h3 = link.find_element(By.XPATH, "./ancestor::*//h3 | .//*//h3 | ./following::h3[1] | ./preceding::h3[1]")
                title = h3.text
            except:
                title = "No title found"
        
        # Find descriptions
        description = "No description found"
        try:
            desc_candidates = [
                link.find_element(By.XPATH, "./ancestor::*[3]//div[string-length(text()) > 50]"),
                link.find_element(By.XPATH, "./following::div[string-length(text()) > 50][1]"),
                link.find_element(By.XPATH, "./ancestor::*//div[contains(@class, 'desc') or contains(@class, 'snippet') or contains(@class, 'description')]")
            ]
            
            for candidate in desc_candidates:
                if candidate and candidate.text.strip():
                    description = candidate.text.strip()
                    break
        except:
            pass  # default value if none
        
        # we only keep rows with at least one non-null value
        if title.strip() and url and "google" not in url.lower():
            search_results.append({
                "Title": title,
                "URL": url,
                "Description": description
            })
    except Exception as e:
        print(f"Error when extracting: {str(e)[:100]}...")
        continue

# No duplicates
unique_results = []
seen_urls = set()
for result in search_results:
    if result["URL"] not in seen_urls:
        seen_urls.add(result["URL"])
        unique_results.append(result)

search_results = unique_results

# Display first results
print(f"\n{Fore.YELLOW}Looking at firt results ({len(search_results)} au total):{Style.RESET_ALL}")
for i, result in enumerate(search_results[:3], 1):
    print(f"\nRésultat {i}:")
    print(f"Title: {result['Title']}")
    print(f"URL: {result['URL']}")
    print(f"Description: {result['Description'][:100]}...")

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = f"google_search_results_{timestamp}.csv"

# Writing in csv
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Title', 'URL', 'Description']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for result in search_results:
        writer.writerow(result)

print(f"\n{Fore.GREEN}Exported results in {csv_filename} ({len(search_results)} entrées){Style.RESET_ALL}")

print("\nExportation terminée!")
for i in tqdm(range(5), desc="Finalisation"):
    time.sleep(0.3)


# I - Webscrapping IMDB (Beautiful Soup)

## A - Making a request on the Top 250 movies page

In [8]:
# looking at the robots.txt
import requests

robots_url = "https://www.imdb.com/robots.txt"
response = requests.get(robots_url)
print(response.text)


# robots.txt for https://www.imdb.com properties
# See https://m.imdb.com/robots.txt for the mDot equivalent
 
User-agent: *
Disallow: /OnThisDay
Disallow: /*/OnThisDay
Disallow: /ads/
Disallow: /*/ads/
Disallow: /ap/
Disallow: /*/ap/
Disallow: /mymovies/
Disallow: /*/mymovies/
Disallow: /r/
Disallow: /*/r/
Disallow: /register
Disallow: /*/register
Disallow: /registration/
Disallow: /*/registration/
Disallow: /search/name-text
Disallow: /*/search/name-text
Disallow: /search/title-text
Disallow: /*/search/title-text
Disallow: /find
Disallow: /*find
Disallow: /find$
Disallow: /*/find$
Disallow: /find/
Disallow: /*/find/
Disallow: /tvschedule
Disallow: /*/tvschedule
Disallow: /updates
Disallow: /*/updates
Disallow: /watch/_ajax/option
Disallow: /*/watch/_ajax/option
Disallow: /_json/video/mon
Disallow: /*/_json/video/mon
Disallow: /_json/getAdsForMediaViewer/
Disallow: /*/_json/getAdsForMediaViewer/
Disallow: /list/ls*/_ajax
Disallow: /*/list/ls*/_ajax
Disallow: /list/ls*/export
Disallow:

In [9]:
# make a request to the IMDB Top 250 Movies page
import requests

url = "https://www.imdb.com/chart/top/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9"
}

# Make the HTTP request
response = requests.get(url, headers=headers)

print(f"request status code: {response.status_code}")
if response.status_code == 200:
    print("all good")
else:
    print("no webpage found")
print('')


request status code: 200
all good



In [10]:
# creating a Beautiful Soup object based on the response.text (the HTML we parsed from the URL)

from bs4 import BeautifulSoup

# Create BeautifulSoup object to parse the HTML
soup = BeautifulSoup(response.text, 'html.parser')


## B - Starting small - retrieving the information of the first movie

In [11]:
# soup.select_one() to fetch the first instance of the element we want to fetch

# Fetch the first movie
first_movie = soup.select_one('h3.ipc-title__text:not(:contains("IMDb Charts"))').text
print(first_movie)

/Users/enzoberreur/Library/Python/3.9/lib/python/site-packages/soupsieve/css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


1. The Shawshank Redemption


In [12]:
first_title = soup.select_one('h3.ipc-title__text:not(:contains("IMDb Charts"))').text.split('. ')[1]
print(first_title)

The Shawshank Redemption


In [19]:
# Trouver tous les spans de la bonne classe
spans = soup.select('div.cZkKPy span.jttFlJ')

first_year = None

# Parcourir les spans et chercher le premier qui contient 199/200/201
for span in spans:
    if "199" in span.text or "200" in span.text or "201" in span.text:
        first_year = span.text
        break

print(first_year)


None


In [16]:
# fetch the first rating

first_rating = soup.select_one('span.ipc-rating-star--rating').text
print(first_rating)


9.3


## C - Retrieve data for the whole page

In [20]:
# initializing lists to store the data

movie_titles = []
movie_years = []
movie_ratings = []

# Get all movie titles
titles = soup.select('h3.ipc-title__text:not(:contains("IMDb Charts"))')
movie_titles = [title.text.split('. ')[1] if '. ' in title.text else title.text for title in titles]
for title in titles:
    movie_titles.append(title.text)

# Get all movie years 
years = soup.select('div.cZkKPy span.jttFlJ:contains("199"), div.cZkKPy span.jttFlJ:contains("200"), div.cZkKPy span.jttFlJ:contains("201")')
for year in years:
    movie_years.append(year.text)

# Get all movie ratings
ratings = soup.select('span.ipc-rating-star--rating')
for rating in ratings:
    movie_ratings.append(rating.text)

print('titles:', movie_titles)
print('years:', movie_years)
print('ratings:', movie_ratings)


titles: ['The Shawshank Redemption', 'The Godfather', 'The Dark Knight', 'The Godfather Part II', '12 Angry Men', 'The Lord of the Rings: The Return of the King', "Schindler's List", 'Pulp Fiction', 'The Lord of the Rings: The Fellowship of the Ring', 'The Good, the Bad and the Ugly', 'Forrest Gump', 'The Lord of the Rings: The Two Towers', 'Fight Club', 'Inception', 'Star Wars: Episode V - The Empire Strikes Back', 'The Matrix', 'Goodfellas', 'Interstellar', "One Flew Over the Cuckoo's Nest", 'Se7en', "It's a Wonderful Life", 'The Silence of the Lambs', 'Seven Samurai', 'Saving Private Ryan', 'City of God', 'Recently viewed', '1. The Shawshank Redemption', '2. The Godfather', '3. The Dark Knight', '4. The Godfather Part II', '5. 12 Angry Men', '6. The Lord of the Rings: The Return of the King', "7. Schindler's List", '8. Pulp Fiction', '9. The Lord of the Rings: The Fellowship of the Ring', '10. The Good, the Bad and the Ugly', '11. Forrest Gump', '12. The Lord of the Rings: The Two T

## D - Creating a dataframe and saving the data

In [21]:
# creating the dataframe

# Take the minimum length to ensure arrays match
min_length = min(len(movie_titles), len(movie_years), len(movie_ratings))

movies_df = pd.DataFrame({
    'Title': movie_titles[:min_length],
    'Year': movie_years[:min_length], 
    'Rating': movie_ratings[:min_length]
})

movies_df.head(5)


,Title,Year,Rating


In [22]:
# saving the dataframe in a dedicated folder

path = 'data'
file = 'imdb_top_movies.csv'

os.makedirs(path, exist_ok=True)
movies_df.to_csv(f'{path}/{file}', index=False)
print(f"Data exported to {path}/{file}")


Data exported to data/imdb_top_movies.csv


# II - Webscrapping an Olist Website

## A - Fetch one product

In [23]:
# Chrome configuration in headless mode
options = Options()
options.add_argument('--headless') # comment if you want to see what the browser is doing
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

# Install Chrome driver
service = Service(ChromeDriverManager().install())

# creating a browser with a driver object
driver = webdriver.Chrome(service=service, options=options)

url = "https://www.sunsetcosmeticos.com.br/produto/base-cobertura-total-maple-absolute-new-york-208"

driver.get(url)

# Wait for elements to be present
wait = WebDriverWait(driver, 10)

print(f"URL: {url}")

# close browser
driver.quit()


URL: https://www.sunsetcosmeticos.com.br/produto/base-cobertura-total-maple-absolute-new-york-208


In [24]:
driver = webdriver.Chrome(service=service, options=options)
time.sleep(2)

driver.set_page_load_timeout(30)
driver.get(url)
print("Connection successful!")
  

Connection successful!


In [25]:
# transforming it into a function

def fetch_product(driver, url):
    '''
    fetches a product's title, price, stars and description
    '''
    # Navigate to URL and get page content
    driver.set_page_load_timeout(30)
    time.sleep(5)
    driver.get(url)
    
    title = driver.find_element(By.CSS_SELECTOR, '.name').text
    price = driver.find_element(By.CSS_SELECTOR, '.cmp-price-price').text
    description = driver.find_element(By.CSS_SELECTOR, '.text').text
    print(f"Title: {title}")
    print(f"Price: {price}") 
    print(f"Description: {description}")

print(fetch_product(driver,url))


Title: Base Cobertura Total - Maple - Absolute New York
Price: R$ 56,90
Description: Base líquida cremosa que oferece cobertura total para um acabamento liso e perfeito. A fórmula sem óleo fornece um acabamento matte que dura por até 12 horas.
Projetado com uma ponta conta-gotas para uma aplicação fácil e sem bagunça.
None


## B - Fetch one brand

In [31]:
# start by fetching the url of the first product of the page, then use the function defined above to fetch its information

# Chrome configuration in headless mode
options = Options()
options.add_argument('--headless') # comment if you want to see what the browser is doing
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

# Install Chrome driver
service = Service(ChromeDriverManager().install())

# creating a browser with a driver object
driver = webdriver.Chrome(service=service, options=options)

url = 'https://www.sunsetcosmeticos.com.br/absolute_new_york'

# Navigate to the brand page
driver.get(url)
print(f"Navigated to brand page: {url}")
time.sleep(5)  # Wait for the page to load

# fetch the url of the first product
try:
    # Find the first product link
    first_product = driver.find_element(By.CSS_SELECTOR, '.product-block a')
    first_product_url = first_product.get_attribute('href')
    print(f"Found first product URL: {first_product_url}")
    
    # fetch its information
    def fetch_product(driver, url):
        '''
        fetches a product's title, price, stars and description
        '''
        try:
            # Navigate to URL and get page content
            driver.set_page_load_timeout(30)
            driver.get(url)
            time.sleep(5)  # Wait for page to load
            
            # Extract product information
            title = driver.find_element(By.CSS_SELECTOR, '.name').text
            price = driver.find_element(By.CSS_SELECTOR, '.cmp-price-price').text
            
            # Try to get description, handle if not available
            try:
                description = driver.find_element(By.CSS_SELECTOR, '.text').text
            except:
                description = "No description available"
            
            # Return as a dictionary
            return {
                'title': title,
                'price': price,
                'description': description,
                'url': url
            }
        except Exception as e:
            print(f"Error fetching product: {str(e)}")
            return None
    
    # Get the product information using the function
    product_info = fetch_product(driver, first_product_url)
    
    if product_info:
        print("\nProduct information:")
        print(f"Title: {product_info['title']}")
        print(f"Price: {product_info['price']}")
        print(f"Description: {product_info['description'][:100]}...")
    else:
        print("Failed to retrieve product information")
        
except Exception as e:
    print(f"Error finding product: {str(e)}")

# Close the browser when done
driver.quit()

Navigated to brand page: https://www.sunsetcosmeticos.com.br/absolute_new_york
Found first product URL: https://www.sunsetcosmeticos.com.br/produto/base-cobertura-total-maple-absolute-new-york-208

Product information:
Title: Base Cobertura Total - Maple - Absolute New York
Price: R$ 56,90
Description: Base líquida cremosa que oferece cobertura total para um acabamento liso e perfeito. A fórmula sem ó...


In [ ]:
def get_all_brands(driver, url='https://www.sunsetcosmeticos.com.br/p/marcas'):
    """
    Fetches all the brands url links present in this url: https://www.sunsetcosmeticos.com.br/p/marcas
    Uses the previous functions to get all of their products urls, and their product information
    """
    print(f"Navigating to brands page: {url}")
    driver.get(url)
    time.sleep(5)  # Wait for the page to load
    
    # Find all brand links
    brand_elements = driver.find_elements(By.CSS_SELECTOR, '.brands-list a')
    brand_urls = [element.get_attribute('href') for element in brand_elements]
    
    print(f"Found {len(brand_urls)} brand URLs")
    
    all_brands_data = []  # Will store data for all brands
    
    # Process each brand
    for i, brand_url in enumerate(brand_urls, 1):
        try:
            print(f"\n{'='*50}")
            print(f"Processing brand {i}/{len(brand_urls)}: {brand_url}")
            
            # Extract brand name from URL
            brand_name = brand_url.split('/')[-1]
            
            # Fetch products for this brand
            print(f"Fetching products for {brand_name}...")
            products = fetch_brand_products(driver, brand_url)
            
            if products:
                print(f"Successfully fetched {len(products)} products for {brand_name}")
                
                # Create DataFrame for this brand
                brand_df = pd.DataFrame(products)
                
                # Add brand name to the DataFrame
                brand_df['brand'] = brand_name
                
                # Save this brand's data
                os.makedirs('data', exist_ok=True)
                file_name = f"data/{brand_name.replace('-', '_')}.csv"
                brand_df.to_csv(file_name, index=False)
                print(f"Saved {len(products)} products to {file_name}")
                
                # Add to the overall collection
                all_brands_data.append({
                    'brand': brand_name,
                    'url': brand_url,
                    'products_count': len(products)
                })
            else:
                print(f"No products found for {brand_name}")
                
            # Be nice to the server
            time.sleep(3)
            
        except Exception as e:
            print(f"Error processing brand {brand_url}: {str(e)}")
            continue
    
    # Create summary DataFrame of all brands
    brands_summary = pd.DataFrame(all_brands_data)
    brands_summary.to_csv('data/all_brands_summary.csv', index=False)
    
    print("\n" + "="*50)
    print(f"Completed processing {len(all_brands_data)} brands")
    print(f"Total products: {brands_summary['products_count'].sum()}")
    print("Summary saved to data/all_brands_summary.csv")
    
    return brands_summary


In [62]:
# create a function to fetch all products of the page (and add a scroll this time, to get them all)

def scroll_and_load_more(driver, max_scrolls=20):
    '''
    scrolls the page with a max of 20 scrolls to load more products
    we count the number of products in the page before scrolling
    we compare this number to the count of products after scrolling
    if we don't have any change, we stop the scroll
    '''
    previous_count = len(driver.find_elements(By.CLASS_NAME, "product-block"))
    scroll_count = 0
    
    while scroll_count < max_scrolls:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait to load more products
        time.sleep(2)
        
        # Count products after scrolling
        current_count = len(driver.find_elements(By.CLASS_NAME, "product-block"))
        
        # If no new products were loaded, break the loop
        if current_count == previous_count:
            print(f"No more products loaded after {scroll_count+1} scrolls. Total products: {current_count}")
            break
            
        print(f"Scroll {scroll_count+1}: Products count increased from {previous_count} to {current_count}")
        previous_count = current_count
        scroll_count += 1
        
    return previous_count

def fetch_brand_products(driver, brand_url, limit=5):
    '''
    Fetches all products of a page (limited to first "limit" products).
    '''
    if not brand_url.startswith('http'):
        brand_url = f'https://www.sunsetcosmeticos.com.br/{brand_url}'
    
    print(f"Navigating to brand page: {brand_url}")
    driver.get(brand_url)
    time.sleep(5)  # Wait for page to load

    # Scroll to load products
    total_products = scroll_and_load_more(driver)
    print(f"Found {total_products} products on the page")

    # Get product URLs
    product_elements = driver.find_elements(By.CSS_SELECTOR, ".product-block a")
    product_urls = [elem.get_attribute('href') for elem in product_elements if elem.get_attribute('href')]

    # Remove duplicates
    product_urls = list(dict.fromkeys(product_urls))
    print(f"Collected {len(product_urls)} unique product URLs")

    # LIMIT to the first 5 products
    product_urls = product_urls[:limit]

    products_data = []

    # Process each product
    for i, product_url in enumerate(product_urls, 1):
        try:
            print(f"Processing product {i}/{len(product_urls)}: {product_url}")
            product_info = fetch_product(driver, product_url)
            
            if product_info:
                products_data.append(product_info)
                print(f"Added product: {product_info['title']}")
            else:
                print(f"Failed to fetch product data from {product_url}")
                
            time.sleep(1)  # polite delay
            
        except Exception as e:
            print(f"Error processing product {product_url}: {str(e)}")
            continue

    print(f"Successfully collected data for {len(products_data)} products")
    return products_data


In [41]:
# testing the code

# Chrome configuration in headless mode
options = Options()
options.add_argument('--headless')  # comment if you want to see what the browser is doing
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

# Install Chrome driver
service = Service(ChromeDriverManager().install())

# Creating a browser with a driver object
driver = webdriver.Chrome(service=service, options=options)

# Set the brand to scrape
brand = 'absolute_new_york'
print(f"Starting to fetch products for brand: {brand}")

# Fetch products for the brand
new_york = fetch_brand_products(driver, brand)
print(f"Successfully fetched {len(new_york)} products")

# Create a DataFrame from the fetched data
df = pd.DataFrame(new_york)

# Add brand column
df['brand'] = brand

# Display the first 5 rows
print("\nFirst 5 products:")
display(df.head(5))

# Close the browser
driver.quit()

Starting to fetch products for brand: absolute_new_york
Navigating to brand page: https://www.sunsetcosmeticos.com.br/absolute_new_york
Scroll 1: Products count increased from 23 to 43
Scroll 2: Products count increased from 43 to 63
Scroll 3: Products count increased from 63 to 83
Scroll 4: Products count increased from 83 to 103
Scroll 5: Products count increased from 103 to 123
Scroll 6: Products count increased from 123 to 143
Scroll 7: Products count increased from 143 to 163
Scroll 8: Products count increased from 163 to 171
No more products loaded after 9 scrolls. Total products: 171
Found 171 products on the page
Collected 171 unique product URLs
Processing product 1/171: https://www.sunsetcosmeticos.com.br/produto/base-cobertura-total-maple-absolute-new-york-208
Added product: Base Cobertura Total - Maple - Absolute New York
Processing product 2/171: https://www.sunsetcosmeticos.com.br/produto/base-cobertura-total-tiramisu-absolute-new-york-207
Added product: Base Cobertura To

,title,price,description,url,brand
0,Base Cobertura Total - Maple - Absolute New York,"R$ 56,90",Base líquida cremosa que oferece cobertura tot...,https://www.sunsetcosmeticos.com.br/produto/ba...,absolute_new_york
1,Base Cobertura Total - Tiramisu - Absolute New...,"R$ 56,90",Base líquida cremosa que oferece cobertura tot...,https://www.sunsetcosmeticos.com.br/produto/ba...,absolute_new_york
2,Base Cobertura Total - Truffle - Absolute New ...,"R$ 56,90",Base líquida cremosa que oferece cobertura tot...,https://www.sunsetcosmeticos.com.br/produto/ba...,absolute_new_york
3,Base Cobertura Total - Mahogany - Absolute New...,"R$ 56,90",Base líquida cremosa que oferece cobertura tot...,https://www.sunsetcosmeticos.com.br/produto/ba...,absolute_new_york
4,Base Cobertura Total - Ganache - Absolute New ...,"R$ 56,90",Base líquida cremosa que oferece cobertura tot...,https://www.sunsetcosmeticos.com.br/produto/ba...,absolute_new_york


In [42]:
# saving the dataframe in the folder you want

file_name = brand.replace('-', '_') + '.csv'
print(file_name)
path = 'data'
df.to_csv(f'{path}/{file_name}', index=False)
print(f"Data saved to {file_name} ({len(new_york)} products)")


absolute_new_york.csv
Data saved to absolute_new_york.csv (171 products)


## C - All brands

# It's too long so i retrieve the first product only


In [63]:
def get_all_brands(driver, url='https://www.sunsetcosmeticos.com.br/p/marcas', product_limit=5):
    """
    Fetches all brand slugs from the main brands page.
    For each brand, fetches products and limits to first N products.
    Saves each brand's products to CSV.
    """
    print(f"Navigating to brands page: {url}")
    driver.get(url)
    time.sleep(5)  # wait for page to load

    # Find all brand divs
    brand_elements = driver.find_elements(By.CSS_SELECTOR, 'div.tag[data-item-name]')
    brand_slugs = [element.get_attribute('data-item-name') for element in brand_elements]

    print(f"Found {len(brand_slugs)} brands.")

    all_brands_data = []  # Will store summary for all brands

    for i, slug in enumerate(brand_slugs, 1):
        try:
            brand_url = f"https://www.sunsetcosmeticos.com.br/{slug}"
            print(f"\n{'='*50}")
            print(f"[{i}/{len(brand_slugs)}] Fetching brand: {brand_url}")

            # Fetch all products normally
            products = fetch_brand_products(driver, brand_url, limit=1)

            # LIMIT the products AFTER fetching
            products = products[:product_limit]

            if products:
                # Create DataFrame
                brand_df = pd.DataFrame(products)
                brand_df['brand'] = slug

                # Save brand products to CSV
                os.makedirs('data', exist_ok=True)
                filename = f"data/{slug.replace('-', '_')}.csv"
                brand_df.to_csv(filename, index=False)
                print(f"Saved {len(products)} products to {filename}")

                # Add to summary
                all_brands_data.append({
                    'brand': slug,
                    'products_count': len(products),
                    'url': brand_url
                })
            else:
                print(f"No products found for brand: {slug}")

            time.sleep(2)  # be nice to server

        except Exception as e:
            print(f"Error processing {slug}: {e}")
            continue

    # Save summary
    summary_df = pd.DataFrame(all_brands_data)
    summary_file = "data/all_brands_summary.csv"
    summary_df.to_csv(summary_file, index=False)
    print(f"\nSaved brand summary to {summary_file}")

    return summary_df


In [64]:
# Chrome setup
options = Options()
options.add_argument('--headless')  
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Start full scraping
print("\n🚀 Starting full SunsetCosmeticos scraping...")
brands_summary = get_all_brands(driver)

print("\n✅ All brands have been processed.")
print(brands_summary)

# Close browser
driver.quit()



🚀 Starting full SunsetCosmeticos scraping...
Navigating to brands page: https://www.sunsetcosmeticos.com.br/p/marcas
Found 61 brands.

[1/61] Fetching brand: https://www.sunsetcosmeticos.com.br/absolute_new_york
Navigating to brand page: https://www.sunsetcosmeticos.com.br/absolute_new_york
Scroll 1: Products count increased from 23 to 43
Scroll 2: Products count increased from 43 to 63
Scroll 3: Products count increased from 63 to 83
Scroll 4: Products count increased from 83 to 103
Scroll 5: Products count increased from 103 to 123
Scroll 6: Products count increased from 123 to 143
Scroll 7: Products count increased from 143 to 163
Scroll 8: Products count increased from 163 to 171
No more products loaded after 9 scrolls. Total products: 171
Found 171 products on the page
Collected 171 unique product URLs
Processing product 1/1: https://www.sunsetcosmeticos.com.br/produto/base-cobertura-total-maple-absolute-new-york-208
Added product: Base Cobertura Total - Maple - Absolute New York